In [1]:
from py2neo import Graph, Path
from py2neo import Node, Relationship
from pymongo import MongoClient
import json

config = json.load(open("config.json"))

graph = Graph("http://"+config["user"]+":"+config["password"]+"@"+config["server"]+":7474")

client = MongoClient()
client = MongoClient(config["server"], 27017)
db = client.KG

## First add all nodes into neo4j graph

In [2]:
def addnodes():
    nodes = db.Node.find({})
    for node in nodes:
        graphnode = Node(node['category'],title = node['label'],content = node['abstract'])
        graph.create(graphnode)
        

addnodes()

In [3]:
query_contents = json.load(open("query_content.json"))
def addedges():
    nodes = db.Node.find({})
    for node in nodes:
        #The origin node in the graph
        for relate in query_contents[node['category']]:
            if relate != "abstract" and relate != "label" and len(node[relate]) > 0:
                for r in node[relate]:
                    query = 'MATCH (c) WHERE c.title=["' + r + '"] RETURN c'
                    if len(graph.run(query).data()) == 0:
                        abstract = ['This is the '+relate+'of node '+node['label'][0]]
                        gn = Node(relate,title = [r],content = abstract)
                        graph.create(gn)
                        cat = relate
                    else:
                        cat = str(graph.run(query).data()[0]['c']).split(':')[1].split(' ')[0]
                    createR = 'MATCH (p:'+node['category']+' {title:["'+node['label'][0]+'"]}),'+\
                    '(c:'+cat+' {title:["'+r+'"]})' +'CREATE (c)-[:`'+relate+'`]->(p)'
                    graph.run(createR)
            else:
                pass

In [4]:
addedges()

In [ ]:
query = 'MATCH (c) WHERE c.title=["' +"Artificial cardiac pacemaker" + '"] RETURN c'
str(graph.run(query).data()[0]['c']).split(':')[1].split(' ')[0]

In [ ]:
gn = Node('label',title = ["Artificial cardiac pacemaker"],content = 'abstract')
graph.run(query).data()